In [ ]:
  ! pip install datasets transformers sacrebleu torch sentencepiece transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 59.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [ ]:
import os
os.environ["WANDB_DISABLED"]="true"

In [ ]:
import transformers
print(transformers.__version__)

4.26.1


In [ ]:
model_checkpoint = "t5-small"

In [ ]:
from datasets import load_dataset, load_metric
raw_datasets = load_dataset("TenzinKhorloVIT/wmt_14_en_fr")
metric = load_metric("sacrebleu")

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset wmt_14_en_fr downloaded and prepared to /root/.cache/huggingface/datasets/TenzinKhorloVIT___wmt_14_en_fr/en-fr/1.0.0/f24a05e7c9e71017a1e2c4eb11aef6cde966251a4de6b116e67256dcd2e0453c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

<ipython-input-5-6b433a797baa>:3: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("sacrebleu")


In [ ]:
import datasets
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))
show_random_elements(raw_datasets["train"])

,translation
0,"{'en': 'On behalf of Parliament, I should like to thank four colleagues from our House who led this work and who made substantial contributions: Mr Gargani, Mr Swoboda, Mrs Frassoni and Mr Clegg.', 'fr': 'Au nom du Parlement, je voudrais remercier quatre de mes collègues qui ont dirigé ces travaux en y apportant une contribution substantielle, à savoir M. Gargani, M. Swoboda, Mme Frassoni et M. Clegg.'}"
1,"{'en': 'I hope that once ratification is concluded there will be nothing to stop us from engaging in a Europe of results, from taking more responsibility for global security, from having a more active impact on neighbouring countries, and finally from discussing EU enlargement, from winning in terms of competition, investments and growth.', 'fr': 'J'espère que, après la ratification, rien ne nous empêchera de nous engager pour une Europe des résultats, d'assumer davantage de responsabilité par rapport à la sécurité globale, d'exercer une influence plus active sur les pays voisins, et enfin de débattre de l'élargissement de l'UE, de gagner en termes de concurrence, d'investissements et de croissance.'}"
2,"{'en': 'This strike is having disastrous consequences for Madagascar' s economy, and last week cost between USD 8-10 million.', 'fr': 'Cette grève a des conséquences désastreuses pour l'économie du pays.'}"
3,"{'en': 'An operation to reduce the debt of 23 countries by USD 50 billion, which is 70% of the total debt, was launched.', 'fr': 'On a aussi parlé de la réduction de la dette de 23 pays, pour un montant de 50 milliards de dollars, soit 70 % du total.'}"
4,"{'en': 'This proposal is under examination by the Council and the European Parliament.', 'fr': 'Cette proposition est à l'examen du Conseil et du Parlement européen.'}"


In [ ]:
fake_preds = ["hello there", "general kenobi"]
fake_labels = [["hello there"], ["general kenobi"]]
metric.compute(predictions=fake_preds, references=fake_labels)

{'score': 0.0,
 'counts': [4, 2, 0, 0],
 'totals': [4, 2, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 1.0,
 'sys_len': 4,
 'ref_len': 4}

In [ ]:
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [ ]:
tokenizer(["Hello, this one sentence!", "This is another sentence."])

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}

In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

{'input_ids': [[8774, 6, 48, 80, 7142, 55, 1], [100, 19, 430, 7142, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1]]}


/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [ ]:
prefix = ""
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "fr"
def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[419, 4078, 102, 1575, 13, 8, 2363, 1], [27, 15884, 4258, 26, 8, 2363, 13, 8, 1611, 12876, 19181, 1211, 29, 15, 26, 30, 1701, 1003, 1882, 5247, 6, 11, 27, 133, 114, 728, 541, 12, 1663, 25, 3, 9, 1095, 126, 215, 16, 8, 897, 24, 25, 2994, 3, 9, 8714, 15723, 1059, 5, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[7144, 7854, 20, 50, 2363, 1], [1022, 30474, 21504, 50, 2363, 146, 2180, 3335, 18779, 285, 3, 3925, 1366, 1413, 3522, 4987, 15, 90, 17438, 1003, 12987, 4635, 3, 15, 17, 528, 327, 20234, 109, 1739, 140, 7, 9056, 226, 3, 35, 16159, 154, 3569, 238, 327, 3, 3618, 8846, 20, 4079, 7, 13941, 5, 1]]}

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/1000000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3003 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-{source_lang}-to-{target_lang}",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True    
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
import numpy as np
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]
    return preds, labels
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation. If translation are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1000000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 62500
  Number of trainable parameters = 60506624
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than usin

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,0.878900,1.123311,9.729900,18.075300


Saving model checkpoint to t5-small-finetuned-en-to-fr/checkpoint-500
Configuration saved in t5-small-finetuned-en-to-fr/checkpoint-500/config.json
Configuration saved in t5-small-finetuned-en-to-fr/checkpoint-500/generation_config.json
Model weights saved in t5-small-finetuned-en-to-fr/checkpoint-500/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-en-to-fr/checkpoint-500/tokenizer_config.json
Special tokens file saved in t5-small-finetuned-en-to-fr/checkpoint-500/special_tokens_map.json
Copy vocab file to t5-small-finetuned-en-to-fr/checkpoint-500/spiece.model
Saving model checkpoint to t5-small-finetuned-en-to-fr/checkpoint-1000
Configuration saved in t5-small-finetuned-en-to-fr/checkpoint-1000/config.json
Configuration saved in t5-small-finetuned-en-to-fr/checkpoint-1000/generation_config.json
Model weights saved in t5-small-finetuned-en-to-fr/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in t5-small-finetuned-en-to-fr/checkpoint-1000/tokenizer_co

TrainOutput(global_step=62500, training_loss=0.8909371772460938, metrics={'train_runtime': 14013.6096, 'train_samples_per_second': 71.359, 'train_steps_per_second': 4.46, 'total_flos': 1.959033665539277e+16, 'train_loss': 0.8909371772460938, 'epoch': 1.0})

In [ ]:
import os
for dirname, _, filenames in os.walk('t5-small-finetuned-en-to-fr/checkpoint-62500'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

t5-small-finetuned-en-to-fr/checkpoint-62500/special_tokens_map.json
t5-small-finetuned-en-to-fr/checkpoint-62500/training_args.bin
t5-small-finetuned-en-to-fr/checkpoint-62500/spiece.model
t5-small-finetuned-en-to-fr/checkpoint-62500/generation_config.json
t5-small-finetuned-en-to-fr/checkpoint-62500/pytorch_model.bin
t5-small-finetuned-en-to-fr/checkpoint-62500/config.json
t5-small-finetuned-en-to-fr/checkpoint-62500/scheduler.pt
t5-small-finetuned-en-to-fr/checkpoint-62500/tokenizer.json
t5-small-finetuned-en-to-fr/checkpoint-62500/trainer_state.json
t5-small-finetuned-en-to-fr/checkpoint-62500/optimizer.pt
t5-small-finetuned-en-to-fr/checkpoint-62500/rng_state.pth
t5-small-finetuned-en-to-fr/checkpoint-62500/tokenizer_config.json


In [ ]:
trainer.save_model(os.getcwd())

Saving model checkpoint to /content
Configuration saved in /content/config.json
Configuration saved in /content/generation_config.json
Model weights saved in /content/pytorch_model.bin
tokenizer config file saved in /content/tokenizer_config.json
Special tokens file saved in /content/special_tokens_map.json
Copy vocab file to /content/spiece.model


In [ ]:
model_name

'fine_tuned_model'

In [ ]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
model_name = 't5-small-finetuned-en-to-fr/checkpoint-62500'
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
output = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]

loading file spiece.model
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading configuration file t5-small-finetuned-en-to-fr/checkpoint-62500/config.json
Model config T5Config {
  "_name_or_path": "t5-small",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "d_ff": 2048,
  "d_kv": 64,
  "d_model": 512,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 6,
  "num_heads": 8,
  "num_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "mi

['Je vais venir à la toilette pour regarder la télévision.']


In [ ]:
output

['Je vais venir à la toilette pour regarder la télévision.']